#### So although we have some outliers in the dataset, I only want to exclude the obvious extreme ones. This basically only means for 'price_usd'. This means I'll leave the other variables as is (for now).

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
df = pd.read_csv('training_set_VU_DM.csv')

In [55]:
pd.set_option('display.max_columns', None)

#### Here you can see that we have no missings for 'price_usd'

In [56]:
# Get the count, number of unique values, and value counts (including missing values) for each column
counts = pd.DataFrame({'count': df.count(), 'unique': df.nunique(), 'missing': df.isnull().sum()})

# Print the result
counts

,count,unique,missing
srch_id,4958347,199795,0
date_time,4958347,198615,0
site_id,4958347,34,0
visitor_location_country_id,4958347,210,0
visitor_hist_starrating,251866,312,4706481
visitor_hist_adr_usd,252988,7799,4705359
prop_country_id,4958347,172,0
prop_id,4958347,129113,0
prop_starrating,4958347,6,0
prop_review_score,4950983,10,7364


#### I set the boundary to 30times the IQR (interquartile range), which is quite extreme.

In [57]:
def detect_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 30 * iqr
    upper_bound = q3 + 30 * iqr
    return (df < lower_bound) | (df > upper_bound)

select_cols= df.select_dtypes(include=['float', 'int']).columns.tolist()

outliers = detect_outliers(df[select_cols])

#### This range finds 3724 outliers for price_usd (so really high prices)

In [58]:
# Count the number of outliers per column
outliers_per_col = outliers.sum()
# Count the total number of outliers
total_outliers = outliers_per_col.sum()

print("Number of outliers per column:")
print(outliers_per_col)
print("Total number of outliers:", total_outliers)

Number of outliers per column:
srch_id                              0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating              0
visitor_hist_adr_usd                 0
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                    0
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2                 0
prop_log_historical_price            0
position                             0
price_usd                         3724
promotion_flag                 1069118
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count              1712113
srch_children_count            1146573
srch_room_count                 438135
srch_saturday_night_bool             0
srch_query_affinity_score            0
orig_destination_distance        

#### But even 30times IQR is still 'only' around 3k. So maybe we should take a broader range.

In [59]:
# used for checking
variable = ['price_usd']
q1 = df[variable].quantile(0.25)
q3 = df[variable].quantile(0.75)
iqr = q3 - q1
iqr*30

price_usd    2998.8
dtype: float64

#### The top 2000 values lie between 19mil and 10k. So to be very subtle, we can set the max to 10k. This is very extreme (since 0.04% of all data is higher than 10k), but it prevents the very high values

In [60]:
print(df['price_usd'].nlargest(2000))

1168566    19726328.00
680748     11818011.00
3117007     9381308.71
1168574     5444467.00
2945135     4973355.00
              ...     
1044575        9999.00
2181331        9999.00
3971118        9994.00
2488437        9992.97
1939330        9987.00
Name: price_usd, Length: 2000, dtype: float64


In [61]:
# Set everything above 10k to missing
df.loc[df['price_usd'] > 10000, 'price_usd'] = np.nan
print(df['price_usd'].nlargest(20))

3186248    10000.00
724587      9999.00
1044575     9999.00
2181331     9999.00
3971118     9994.00
2488437     9992.97
1939330     9987.00
3119351     9982.00
1532566     9975.00
2974351     9970.00
3971124     9948.00
1620742     9929.00
3375984     9924.00
760980      9917.57
1152974     9915.25
3186239     9870.00
70783       9859.00
1110765     9849.69
3407845     9846.00
166822      9836.92
Name: price_usd, dtype: float64


#### So now we see around 2k missings for 'price_usd'

In [62]:
# Get the count, number of unique values, and value counts (including missing values) for each column
counts = pd.DataFrame({'count': df.count(), 'unique': df.nunique(), 'missing': df.isnull().sum()})

# Print the result
counts

,count,unique,missing
srch_id,4958347,199795,0
date_time,4958347,198615,0
site_id,4958347,34,0
visitor_location_country_id,4958347,210,0
visitor_hist_starrating,251866,312,4706481
visitor_hist_adr_usd,252988,7799,4705359
prop_country_id,4958347,172,0
prop_id,4958347,129113,0
prop_starrating,4958347,6,0
prop_review_score,4950983,10,7364


#### Here I create a new csv file

In [63]:
df.to_csv('training_set_VU_DM_PrepOutliers.csv', index=False)